# Описание исследования

Проводим работу с табличными данными, в которых представлена информация о стартапах, которые функционировали в период с 1980 по 2018 годы. Нужно предсказать, какие из них закроются, а какие нет. Соревнование проводится на популярной платформе Kaggle, что позволит не только применять на практике свои знания в области анализа данных и машинного обучения, но и освоить работу с этой платформой. 

# Цель исследования

- Разработать модель машинного обучения для предсказания продолжения деятельности стартапа. 
- Провести полноценный разведочный анализ и сформировать рекомендации будущим создателям стартапов (какие факторы влияют на успешность стартапа).

# Этапы исследования

- загрузка и ознакомление с данными,
- предварительная обработка,
- полноценный разведочный анализ,
- разработка новых синтетических признаков,
- проверка на мультиколлинеарность,
- отбор финального набора обучающих признаков,
- выбор и обучение моделей,
- итоговая оценка качества предсказания лучшей модели,
- анализ важности ее признаков,
- подготовка отчета по исследованию.

# Описание данных

Датасет состоит из двух файлов: 
- тренировочный набор (около 53к записей) и тестовый набор (около 13к записей). 
- Тренировочный набор содержит целевой признак status, указывающий на то, закрылся стартап или продолжает действовать. 
Временной период - '1970-01-01' по '2018-01-01'. 
Дата формирования выгрузки - '2018-01-01'

- `kaggle_startups_train_27042024.csv` - информация (53 000) стартапах, которые будут использоваться в качестве обучающих данных.
- `kaggle_startups_test_27042024.csv` - информация (13 000) стартапах, которые будут использоваться в качестве тестовых данных. Ваша задача - предсказать значение 'status' для каждого стартапа из этого датасета.
- `kaggle_startups_sample_submit_27042024.csv` - файл с примером предсказаний в правильном формате.
name - идентификатор (название) стартапа в тестовом наборе.
status - целевой признак. Для каждого стартапа предскажите категориальное значение соответствующее прогнозу ['operating', 'closed']. 

**Описание полей данных**
- `name` - Название стартапа
- `category_list` - Список категорий, к которым относится стартап
- `funding_total_usd` - Общая сумма финансирования в USD
- `status` - Статус стартапа (закрыт или действующий)
- `country_code` - Код страны
- `state_code` - Код штата
- `region` - Регион
- `city` - Город
- `funding_rounds` - Количество раундов финансирования
- `founded_at` - Дата основания
- `first_funding_at` - Дата первого раунда финансирования
- `last_funding_at` - Дата последнего раунда финансирования
- `closed_at` - Дата закрытия стартапа (если применимо)
- `lifetime` - Время существования стартапа в днях

In [ ]:
! pip install phik -q
! pip install shap -q
! pip install -U scikit-learn -q
! pip install seaborn --upgrade -q
! pip install missingno -q

In [ ]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

In [ ]:
import os
import matplotlib.pyplot as plt
import missingno as msno
import pandas as pd

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", "{:.3f}".format)

In [ ]:
def preprocessing(df):
    print('Изучение данных датафрейма "\n')
    print("Вывод первых 5 строк \n")
    display(df.head(5))
    print("-" * 100)
    print(f"Размерность датафрейма, составляет - {(df.shape)}")
    print("-" * 100)
    print("Общая информация:\n")
    display(df.info())
    print("-" * 100)
    print("Описательная статистика:\n")
    display(df.describe())
    print("-" * 100)
    df.isna().sum()
    print("Проверка на наличие явных дубликатов:\n")
    display(df.duplicated().sum())
    print("-" * 100)
    print("Вывод количества уникальных значений")
    display(pd.DataFrame(df.apply(lambda x: x.nunique())))

In [ ]:
pth1 = "./datasets/kaggle_startups_train_27042024.csv"
pth2 = "./datasets/kaggle_startups_test_27042024.csv"

if os.path.exists(pth1):
    # Тренировочная выборка:
    df_ks_train = pd.read_csv(
        pth1,
        parse_dates=["founded_at", "first_funding_at", "last_funding_at", "closed_at"],
    )
    if os.path.exists(pth2):
        # Тестовая выборка:
        df_ks_test = pd.read_csv(pth2)
    else:
        print("Something is wrong")

Выведем первоначальную информацию о датафрейме с тренировочными данными

In [ ]:
preprocessing(df_ks_train)

Посмотрим количество пропусков в тренировочных данных

In [ ]:
nan_counts = df_ks_train.isna().sum()
print(nan_counts)

In [ ]:
# Создание объекта осей
fig, ax = plt.subplots(figsize=(18, 4))
# Генерация графика с использованием объекта осей
msno.matrix(df_ks_train, ax=ax, sparkline=False)
# Добавление названия графика
ax.set_title("Матрица пропущенных значений")
# Отображение графика
plt.show()

**Что можно сказать из первоначальных данных:**
- `name` - имеет 1 пропуск, думаю его можно оставить, так как данная колонка у нас отвечает за идентификаторы.
- `category_list`   - список категорий -  имеет большое количество пропусков, подумаем что с ними можно сделать  2465, так же иммет огромное количество категорий, но множество из них разделено знаком |, предлагаю удалить данные и оставить название категории до вышеуказанного символа. Позволит уменьшить количество уникальных категорий.
- `funding_total_usd`   - Общая сумма финансирования в USD - тоже множество пропусков  - 10069, предлагаю заполнить медианными значениями. 
- `status`   - статус. Не имеет проблем в данных
- `country_code`     - код старны - имеет много пропусков 5501, чем заполнить пока не ясно, оставим, так же имеет не так много категорий, переведем в категориальный тип.
- `state_code`      state_code - Код штата    имеет много пропусков  6762, чем заполнить пока не ясно, оставим, так же имеет не большое количество категорий, переведем в категориальный тип. 
- `region`        - региона, много пропусков -  6358, и много категорий. Думаю дальше данные эти не понадобятся для обучения модели, оставим пропуски
- `city`           - город, много пропусков -        6358, соответствует данным по региону. Вообще все пропуски связанные с географическим положением не заполнены в одинх и теж же строках, это можно наглядно увидеть по матрице пропусков. Возможно кто то заполнял только код страны, а остальные данные были не обязательны для заполнения, либо данные о них просто отсутствовали 
- `funding_rounds`  -   Количество раундов финансирования    - Не имеет проблем в данных. Только переведем в int, так как даные числовые.
- `founded_at`       -    Дата основания    - Не имеет проблем в данных.
- `first_funding_at`  -     Дата первого раунда финансирования - Не имеет проблем в данных.
- `last_funding_at`    -      Дата последнего раунда финансирования - Не имеет проблем в данных.
- `closed_at`        -    Дата закрытия стартапа (если применимо) - имеет большое количество пропусков - 47599, что может говорит о том, что большинство из представленных в данных стартапов являются действующими. Данная колонка будет мешать предсказанию, так как нам надо именно предсказать будет ли закрыт стартап или нет, поэтому заполним пропуски датой выгрузки данных и создадим новую синтетическую информацию, а именно `lifetime` - Время существования стартапа в днях, для этого посчитаем количество дней прошедших между датой основания и датой закрытия